In [1]:
# Make the libraries reload automatically
# This can be useful if you change something 
# a imported library. 
%load_ext autoreload
%autoreload 2

### Import Libraries

First a series of libraries are loaded which are used within this notebook. 
If any of the libraries is missing, simply install them using `pip`:
    
```lang=python
pip install <library-name>
```

In [2]:
# Import the required libraries
import dotenv
import gmplot                        # Plot data on Google Maps instance 
from IPython.display import IFrame   # Display HTML page within Jupyter cells
import matplotlib.pyplot as plt      # Plotting library
import numpy as np                   # Scientific computing package
import os                            # OS dependent functionalities
import pandas as pd                  # Data analysis package
import statistics                    # Statistic operations package
from stravaio import StravaIO, strava_oauth2
import warnings                      # Warning class


dotenv.load_dotenv()

True

### Before you Continue

To retrieve your strava token, make sure that you have correctly set your Strava account and exported the API client ID and secret. To make life easier, you can them using the following in a terminal:

```lang=bash
export STRAVA_CLIENT_ID=<the-actual-id>
export STRAVA_CLIENT_SECRET=<the-actual-secret>
```

In [3]:
# Retrieve your strava  access token
#token = strava_oauth2(client_id=<the-actual-id>, client_secret=<the-actual-secret> )
#assert access_token is not None
access_token = strava_oauth2(client_id="<the-actual-id>", client_secret="<the-actual-secret>" )
assert access_token is not None

### Retrieve Client Data

Using the Strava access token the client data is stored in the `client` object. Use `print(client)` to print the content of the `client` object.

In [4]:
# Store client data using the retrieved access-token
client = StravaIO(access_token=access_token['access_token'])

### Retrieve Athlete Data

The next line can be used to extract some athlete information. This `athlete` object contains more ore less that what is also stored on your Strava profile page.

In [5]:
# Retrieve athlete data
athlete = client.get_logged_in_athlete()

# Uncomment to print the retrieved athlete data
# print(athlete)

### Retrieve Activities

The next cell will retrieve the activities that are recorded after a certain given date.
The date is stored in `activities_from_this_point_on`

In [6]:
# Retrieve client's activities recorded after the given "date"
activities_from_this_point_on = 'last month'


activities = client.get_logged_in_athlete_activities(after=activities_from_this_point_on,
                                                     list_activities=None)


Fetched 17, the latests is on 2020-02-29 07:15:35+00:00


### Athlete Statistics

The next cell prints some basic activitie data. Statistics such as total runs, ride and distances are printed.

In [7]:
# initialize list of various acitivy types
run_acitivies = []
ride_acitivies = []
virtual_ride_acitivies = []

# sort all relevant activities in corresponding lists
for a in activities:
    if a.type == 'Run':
        run_acitivies = run_acitivies + [a]
    elif a.type == 'Ride':
        ride_acitivies = ride_acitivies + [a]    
    elif a.type == 'VirtualRide':
        virtual_ride_acitivies = virtual_ride_acitivies + [a] 

# Get total distance per activity
def total_distance(activities):
    
    total_distance = 0

    # sum all relevant activity type distances
    for a in activities:
        total_distance = total_distance + a.distance / 1000
        
    return total_distance

# Print total number or activies per activity type  
# see https://pyformat.info/ for python string formatting overview (old and new style)
print("Actiity stats for: {} {}:".format(athlete.api_response.firstname,
                                        athlete.api_response.lastname))
print("==================================================")
#print("Activity types logged: {}".format(i))
print("Logged Activity types:")
print(*np.unique([ a.type for a in activities]), sep = ", ")  
print("==================================================")
print(" Total number of rides:           {:>7}    [-]".format(len(ride_acitivies)))
print(" Total distance of rides:         {:>10.2f} [km]\n".format(total_distance(ride_acitivies)))
print("==================================================")
print(" Total number of virtual rides:   {:>7}    [-]".format(len(run_acitivies)))
print(" Total distance of virtual rides: {:>10.2f} [km]\n".format(total_distance(virtual_ride_acitivies)))
print("==================================================")
print(" Total number of runs:            {:>7}    [-]".format(len(run_acitivies)))
print(" Total distance of runs:          {:>10.2f} [km]".format(total_distance(run_acitivies)))


### Print Activity Map

The next code snippt will take an activity in your `activities` instance, plot its GPS data on a map and 
prints this interactive map.

This code uses the "streams" of an activity, which contains data such as heart rate and gps location over time.

In [8]:
# Specify activity number according to the order in the activities instance
activity_number = 1

# Retrieve "streams" of activity
streams = client.get_activity_streams(athlete_id=athlete.id, id=activities[activity_number].id)

# Generate bounding box for map
BBox = (min(streams.lat), max(streams.lat),      
        min(streams.lng), max(streams.lng))

# Latitute and longitude data of the activity
latitude_list = streams.lat
longitude_list = streams.lng

# Create Google Maps plot of GPS data.
gmap3 = gmplot.GoogleMapPlotter(statistics.mean(BBox[:2]), 
                                statistics.mean(BBox[2:]), 14) 


# Plot method Draw a line in 
# between given coordinates 
gmap3.plot(streams.lat, streams.lng,  
           'cornflowerblue', edge_width = 2.5) 

# Store the generated map as an HTML file.
gmap3.draw( "map13.html" ) 

# Display the 
IFrame(src='./map13.html', width=700, height=600)